In [ ]:
import sys
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
def print_PF_summary(pf):
    print('{:12s} {:>10s} {:>10s}'.format('', 'P [MW]', 'Q [MVAr]'))
    print('-' * 33)
    Ptot,Qtot = 0,0
    coeff = {'SMs': -1, 'SGs': -1, 'loads': 1, 'lines': 1, 'transformers': 1}
    for key in coeff:
        if key in ('lines','transformers'):
            P = coeff[key] * np.sum(list(pf[key]['Ptot'].values()))
            Q = coeff[key] * np.sum(list(pf[key]['Qtot'].values()))
        else:
            P = coeff[key] * pf[key]['Ptot']
            Q = coeff[key] * pf[key]['Qtot']
        Ptot += P
        Qtot += Q
        print('{:<12s} {:10.2f} {:10.2f}'.format(key, P, Q))
    print('=' * 33)
    print('{:12s} {:10.2f} {:10.2f}'.format('TOTAL', Ptot, Qtot))

In [ ]:
config_IDs = (44,17)
blob = {}
data, PF = {}, {}
time, speed = {}, {}
for ID in config_IDs:
    blob[ID] = {}
    data[ID], PF[ID] = {}, {}
    time[ID], speed[ID] = {}, {}
    for suffix,key in zip(('', 'no_'), 'YN'):
        blob[ID][key] = np.load('../{}slack_{}.npz'.format(suffix,ID), allow_pickle=True)
        data[ID][key] = blob[ID][key]['data'].item()
        PF[ID][key] = blob[ID][key]['PF'].item()
        time[ID][key] = blob[ID][key]['time']
        speed[ID][key] = data[ID][key]['gen']['s:xspeed']

In [ ]:
for slack in 'YN':
    for ID in config_IDs:
        print('>>>>> Fiumesanto {} - Slack {} <<<<<'.format('ON' if ID == 44 else 'OFF',
                                                            'ON' if slack == 'Y' else 'OFF'))
        print_PF_summary(PF[ID][slack])
        print()

In [ ]:
ID = 44
print('{:^4s} {:26s} {:>8s} {:>8s} {:>8s} {:>8s}'.format('#', 'NAME', 'P+slack', 'P-slack', 'Q+slack', 'Q-slack'))
print('-' * 67)
for i,sm in enumerate(PF[ID]['Y']['SMs']):
    if 'GEN' in sm:
        sys.stdout.write('[{:2d}] {:26s} '.format(i+1, sm))
        for pq in 'PQ':
            for slack in 'YN':
                sys.stdout.write('{:8.1f} '.format(PF[ID][slack]['SMs'][sm][pq]))
        sys.stdout.write('\n')
print('=' * 67)
print('{:^4s} {:>26s} {:>8.1f} {:>8.1f} {:>8.1f} {:>8.1f}'.format('', 'TOTAL:',
                                                                 PF[ID]['Y']['SMs']['Ptot'],
                                                                 PF[ID]['N']['SMs']['Ptot'],
                                                                 PF[ID]['Y']['SMs']['Qtot'],
                                                                 PF[ID]['N']['SMs']['Qtot']
                                                                ))

In [ ]:
fig,ax = plt.subplots(2, 1, figsize=(6,5), sharex=True)
col = {'Y': 'k', 'N': 'r'}
j = 0
for ID,a in zip(config_IDs,ax):
    for slack in 'YN':
        a.plot(time[ID][slack], speed[ID][slack][:,j], col[slack], lw=1)
        a.set_ylabel('ω [p.u.]')
ax[0].set_title('Fiumesanto OFF')
ax[1].set_title('Fiumesanto ON')
ax[-1].set_xlabel('Time [s]')
sns.despine()
fig.tight_layout()